# Price Prediction based off Zillow Cover Image

Uses VGG16 architecture to predict the price of a house given an image of it.

In [ ]:
# Imports and loading the data

import pandas as pd 
pd.set_option('display.max_columns', None)
data = pd.read_csv("imageFilenames.csv")
data.head()

In [ ]:
from PIL import Image
import numpy as np

# Want to know how we should format the height x width image data dimensions
# for inputting to a keras model
def get_size_statistics(data):
    heights = []
    widths = []
    img_count = 0
    for i, row in data.iterrows():
        if row["imgFilepath"] != "no photo" or row["imgFilepath"] != np.nan:
            try:
                imgData = np.array(Image.open(row["imgFilepath"]))
                heights.append(imgData.shape[0])
                widths.append(imgData.shape[1])
                img_count += 1
            except:
                data.at[i, "imgFilepath"] = "no photo"
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))

get_size_statistics(data)

In [ ]:
# Cleaning the data

noPhotoIndx = data[data["imgFilepath"] == "no photo"].index 

data.drop(noPhotoIndx, inplace=True)

data = data[data["totPurchaseAmt"] >= 50000]
data = data[data["totPurchaseAmt"] <= 5000000]

data = data[data["yearBuilt"] <= 2020]
data = data[data["yearBuilt"] >= 1940]

data = data[data["zipcode"] <= 56763]
data = data[data["zipcode"] >= 55001]

data = data[data["livingArea"] <= 30000]
data = data[data["livingArea"] >= 1000]

data.drop_duplicates("mediumImageLink", keep = "first", inplace = True)

data = data[["totPurchaseAmt", "imgFilepath"]]

data.dropna(inplace=True)


# Data Description

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.head()

# Loading and Resizing Images

In [ ]:
IMG_SIZE = 224

def resizeAndLoadData(data):
    imageData = []
    for i, row in data.iterrows():
        img = Image.open(row["imgFilepath"])
        img = img.resize((IMG_SIZE, IMG_SIZE))
        imageData.append(np.array(img))
    data["imgData"] = imageData

resizeAndLoadData(data)

In [ ]:
data.head(1)

# Create Train and Test Sets

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
X = np.array(data["imgData"])
#y = to_categorical(data["amtBucketNum"], num_classes=len(bucketList)) # Try bucketing, didn't have as good of results
y = data["totPurchaseAmt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X[0]) # Show the first image

In [ ]:
y[0] # Show the image's price

# Neural Network

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [ ]:
from keras import applications

vggModel = Sequential()

vggModel.add(applications.VGG16(weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False)) # Import the VGG network with imagenet weights

vggModel.add(Flatten(input_shape=vggModel.output_shape[1:]))
vggModel.add(Dropout(.1)) # add dropout to prevent overfitting
vggModel.add(Dense(1, activation="linear")) # output price prediction

vggModel.summary()

In [ ]:
vggModel.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae']) # monitor MAE during training process, use MSE loss function

In [ ]:
X_train = np.array([i for i in X_train]).reshape(-1, IMG_SIZE, IMG_SIZE, 3) # Get images in right format


vggModel.fit(X_train, y_train, batch_size=50, epochs=100, verbose=1, validation_split=0.1) # Fit with a 10% validation set

In [ ]:
X_test = np.array([i for i in X_test]).reshape(-1, IMG_SIZE, IMG_SIZE, 3) # Get test images in right format

loss, mae = vggModel.evaluate(X_test, y_test, verbose=1) # Test our model

print("Loss (MSE): ", loss)
print("MAE: ", mae)

In [ ]:
vggModel.save("ZillowCNN.h5") # Save Model